<a href="https://colab.research.google.com/github/khirotaka/Swift4TF/blob/master/Colab/Swift%20for%20TensorFlow%20for%20using%20PyTorch%20users%20No2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 自動微分

In [0]:
import TensorFlow

Context.local.randomSeed = TensorFlowSeed(graph: 0, op: 0)

PyTorchでは、以下のように `requires_grad=True` を使って微分するしないを明記するが、
```python
x = torch.ones(2, 2, requires_grad=True)
```

Swift for TensorFlow では、TensorFlow 2系 の `tf.GradientTape()` に近い書き方をする。

In [3]:
var x = Tensor<Float>(ones: [2, 2])
print(x)

[[1.0, 1.0],
 [1.0, 1.0]]


In [4]:
var y = TensorFlow.valueWithGradient(at: x) { x in      // PyTorch の .backward()
    (x + 2.0).sum()
}

y       // 二つの要素を持つタプル

▿ 2 elements
  - value : 12.0
  - gradient : [[1.0, 1.0],
 [1.0, 1.0]]


戻り値は タプルなので、次のように書けば各要素にアクセスできる。

In [5]:
print("Value: \(y.0)")
print("Gradient: \n \(y.1)")

Value: 12.0
Gradient: 
 [[1.0, 1.0],
 [1.0, 1.0]]


## 導関数を求める

In [0]:
@differentiable
func f(_ inputs: Double) -> Double {
    return inputs * inputs   // x^2
}

let df = gradient(of: f)

In [7]:
f(5.0)

25.0


In [8]:
df(5.0)

10.0


PyTorchの 
```python
with torch.no_grad():
    ...
```

的なやつ

`withoutDerivative(at: )` を使う事で、引数に指定した関数は勾配計算を行わない。

In [9]:
let a = 10.0

gradient(at: a) { a in 
    f(a) + withoutDerivative(at: f(a))  // 2x
}

20.0


## 簡単なニューラルネット

Swift for TensorFlowでのモデル定義は、PyTorchのそれとほとんど同じ。  

In [0]:
@differentiable
func mish(_ inputs: Tensor<Float>) -> Tensor<Float> {
    return inputs * tanh(softplus(inputs))
}

In [0]:
struct Network: Module {
    var fc1 = Sequential(
        Dense<Float>(inputSize: 4, outputSize: 128, activation: mish),
        Dense<Float>(inputSize: 128, outputSize: 64, activation: mish)
    )
    
    var fc2 = Dense<Float>(inputSize: 64, outputSize: 10)

    @differentiable
    func callAsFunction(_ input: Tensor<Float>) -> Tensor<Float> {
        let out = fc1(input)
        return mish(fc2(out))
    }
}

In [0]:
Context.local.learningPhase = .training    // PyTorch の model.train() と同じ

var model = Network()
let optimizer = Adam(for: model, learningRate: 0.01)


let x = Tensor<Float>(randomNormal: [10, 4])
let y = Tensor<Int32>([0, 1, 0, 1, 8, 3, 4, 6, 9, 2])

In [13]:
for _ in 0..<10 {
    let delta_model = gradient(at: model) { model -> Tensor<Float> in
        let pred = model(x)
        let loss = softmaxCrossEntropy(logits: pred, labels: y)
        print(loss)
        return loss
    }
    optimizer.update(&model, along: delta_model)
}

2.32212
2.103107
1.854009
1.6049321
1.3833803
1.1867671
1.0061058
0.8452984
0.7096749
0.59515756
